## Example Experiment:  I-V sweeps at different temperatures

The following code shows an example of a heating and biasing experiment.
This experiment follows these steps:

1. Set a temperature in the temperatures list
2. Wait until the measured temperature has reached the setpoint within 1 degree
3. Start a Voltage sweep (dual) and plot the I-V curve in real-time
4. Set the next temperature in the temperatures list and continue from point 2 until all temperatures have been set. 

Before running this experiment, start a __Heating__ and __Biasing (source mode: Voltage)__ experiment in Impulse.

In [1]:
# Import our libraries
import impulsePy as impulse
import numpy as np
from time import sleep
import matplotlib.pyplot as plt

# The line below opens the graph in a separate window (required for live updating)
%matplotlib qt

# Subscribe to data and event streams
impulse.heat.data.subscribe()
impulse.bias.data.subscribe()

impulse.waitForControl() # Wait until impulse is in control mode
sleep(2) # Wait for some data to come in

Subscribed to impulse events
Subscribed to heat data
Subscribed to bias data
[IMPULSE STATUS]: control


In [2]:
# Create graph window
fig = plt.figure()
fig.canvas.set_window_title('I-V sweep')
ax = fig.add_subplot(111)
ax.set_xlabel("Voltage (V)")
ax.set_ylabel("Current (A)")
plt.ion()
plt.show()

# Experiment parameters
temperatures = [100, 200] #C
startVoltage = -10
stopVoltage = 10
voltageSteps = 20
compliance = 1E-2 #A

voltageSetpoints = np.linspace(startVoltage, stopVoltage, voltageSteps) # Create a list of voltage setpoints
voltageSetpoints = np.append(voltageSetpoints,np.flipud(voltageSetpoints),axis=0) # Add the reversed list to the end for the sweep down

plt.axhline(y=compliance, color='r', linestyle='--', label="Compliance") # Add compliance to the graph as a dotted line

# Experiment code
for temperature in temperatures:
    print(f"Setting the temperature to {temperature} degrees")
    impulse.heat.set(temperature)
    print(f"Waiting for temperatureMeasured to reach {temperature} within 1 degree")
    while abs(impulse.heat.data.getNewData()["temperatureMeasured"]-temperature)>1: # While the temperature is not met (within 1 degree)
        sleep(0.1) # Wait for 0.1 seconds
    print(f"Temperature {temperature} reached, starting sweep...")
    impulse.bias.data.setFlag(f"T{temperature}") # Add a flag to the bias data to be able to recall this cycle later on
    datx=[] # Create an empty list for the x data
    daty=[] # Create an empty list for the y data
    Ln, = ax.plot(datx,daty,label=f"T{temperature}") # Add a new plot to the graph with the temperature as the label
    plt.legend() # Tell the plot to include a legend
    impulse.bias.data.getNewData() # This is to make sure that the next time we ask for NewData we are getting data from after the first voltage setpoint
    for voltage in voltageSetpoints: # Go through all the setpoints for the voltage sweep
        impulse.bias.set(voltage,compliance)
        measurement = impulse.bias.data.getNewData()
        datx.append(measurement["voltageMeasured"])
        daty.append(measurement["currentMeasured"])
        Ln.set_ydata(daty)
        Ln.set_xdata(datx)
        ax.relim()
        ax.autoscale_view()
        plt.pause(0.01)
    print("Sweep finished.")
        
impulse.bias.set(0,compliance) # Set the voltage back to 0
impulse.bias.data.setFlag("") # Set the bias data flag to nothing

Setting the temperature to 100 degrees
Waiting for temperatureMeasured to reach 100 within 1 degree
Temperature 100 reached, starting sweep...
Sweep finished.
Setting the temperature to 200 degrees
Waiting for temperatureMeasured to reach 200 within 1 degree
Temperature 200 reached, starting sweep...
Sweep finished.


In [3]:
impulse.disconnect() # Disconnect from Impulse

Disconnected from impulse
